# Question 2: Classification [20 Marks]

**[NOTE]** For each question, it is possible the your code implementation is correct even if some public test cases seem to fail (due to time-limit, or missing dependencies, or runtime under/overflow). On the hand, even if your code passes all the test cases, you are not guaranteed to be completely correct.

**[Background Story, OK to skip]** Intrusion Detection System (IDS) is a common solution in cybersecurity to protect the network from malicious activities. IDS is a solution which is deployed at strategic points in a network which allows it to monitor the incoming and outgoing traffic in search for suspicious/malicious traffic. A type of IDS is anomaly-based IDS, which uses behavior-based detection. By monitoring inbound and outbound network traffic, IDS continuously conducts network traffic analysis and extracts statistical features of network, and alarms if any suspicious activity is detected.

**[TASK]** For Question 2, you will use the Intrusion Detection Evaluation Dataset (CICIDS2017) published by University of New Brunswick in co-operation with the Canadian Institute for Cybersecurity to design a simple anomaly-based Intrusion Detection System using machine learning. The task of intrusion detection will be modelled as a **multi-class classification**, which you will train a **Softmax Regressor model** using network features for the classification of different types of network traffic (Safe / Different types of Denial of Service (DoS) attacks). 

You are given a processed version of the original dataset, which has been cleaned and filtered down to 10 features from the original 78 features. Your exact understanding of each of the features / DoS Attack will not be tested as part of this question, and you are encouraged to focus on the implementation of your ML model and the evaluation of your results using the given metrics.

Information on the data features and the target variable is as follows:
* Features 
    * Bwd Packet Length Mean
    * Max Packet Length
    * Average Packet Size
    * Packet Length Mean
    * Total Length of Bwd Packets
    * Packet Length Std
    * Total Fwd Packets
    * Bwd Packet Length Std
    * Avg Bwd Segment Size
    * Packet Length Variance
* Target (Classification types of the network)
    * Benign (Harmless network traffic)
    * DoS GoldenEye
    * DoS Hulk
    * DoS Slowhttptest
    * DoS slowloris
        
*('Bwd' in this case refers to the Backwork direction of network traffic, and 'Fwd' Forward direction. 'DoS' refers to Denial of Service)*

In this question, the dataset has already been split into train and test for you.

* Training set data features: X_train
* Testing set data features: X_test
* Training set target classes: y_train
* Testing set target classes: y_test

Important point to note is that the target matrix `y` given to you is in the shape of (n, K), where n=number of data points and K=total number of classes. The target has been encoded into a `one-hot encoded` vector, which is a vector representation where all the elements of the vector are 0 expect one, which has 1 as its value corresponding to the index of the class. As an example, given that the set of classes is `(Benign, DoS Golden Eye, Dos Hulk, DoS Slowhttptest, DoS slowloris)`, if `y = Benign`, y can be represented as the following vector `[1,0,0,0,0]`. Similarly, `y = DoS slowloris` is represented as `[0,0,0,0,1]`. The index of the one-hot encoded vector will follow the ordering of this list `[Benign, DoS Golden Eye, Dos Hulk, DoS Slowhttptest, DoS slowloris]`.

This understanding will aid your calculations for this question.

In [1]:
# required packages
import time
import math
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Import data
X_train = np.load('./X_train.npy')
y_train = np.load('./y_train.npy')
X_test = np.load('./X_test.npy')
y_test = np.load('./y_test.npy')

# label index of the one hot encoded vector - you may use this as a reference (or as part of your code) when you evaluate 
# your model using calculated metrics. Otherwise, it is not needed.

mapping = {
    0:'Benign',
    1:'DoS Golden Eye',
    2:'DoS Hulk',
    3:'DoS Slowhttptest',
    4:'DoS slowloris'
}

*Each sub-question of Question 2 is designed to be graded independently from other parts, hence students are encouraged to attempt as many questions as possible (if not all).*

### A. Softmax Regression Hypothesis [3 marks]

Softmax Regression, more commonly known as multinomial logistic regression, is a generalization of the logistic regression model to multi-class classification settings. By combining your concepts from the logistic regression and the softmax function that you have learnt in CS2109S, you will be able to implement this classfier.

In logistic regression, we defined the following hypothesis:

$$ h_{w}(x) = \frac{1}{1+exp(-w^{\intercal}x)}$$

and the corresponding cost function:

$$ J(w) = - \left[ \sum_{i=1}^{m} y^{(i)}\log{h_{w}(x^{(i)})} + (1-y^{(i)})\log{h_{w}(x^{(i)})} \right] $$

We have also learnt about the softmax function, which converts a vector of $K$ real numbers into a probability distribution of $K$ different outcomes, defined as:

$$ P(y=j) = \frac{e^{z_{j}}}{\sum_{i=0}^{k}e^{z_{i}}}  $$

In softmax regression, we will formulate a hypothesis where given a test input $x$, we estimate the probability that $P(y=k|x)$ for each class of $k=1,...,K$. In other words, we want to estimate the probability of the class label taking on each of the $K$ different possible values. This will be done by using the softmax function and our hypothesis will be defined as :

$$ \begin{align}
h_{w}(x) =
\begin{bmatrix}
P(y = 1 | x; w) \\
P(y = 2 | x; w) \\
\vdots \\
P(y = K | x; w)
\end{bmatrix}
=
\frac{1}{ \sum_{j=1}^{K}{\exp(w^{(j)\top} x) }}
\begin{bmatrix}
\exp(w^{(1)\top} x ) \\
\exp(w^{(2)\top} x ) \\
\vdots \\
\exp(w^{(K)\top} x ) \\
\end{bmatrix}
\end{align}  $$

To calculate the above hypothesis, we will be using a weight matrix of shape (n,K), where n is the number of data features and K is the total number of classes. Thus, each column $j$ of the weight matrix will be used for the calculation of class $j$, for $j=1,..,K$ different classes.  

Lastly, we will ensure that our calculation of the exponential term do not cause numerical overflow during training. 

Define a function `softmax_regression_hypothesis` which implements this hypothesis function. 

Hint: Where you need to use the exponential function in numpy, e.g. `np.exp(z)`, you can use `np.exp(z - np.max(z, axis=1, keepdims=True))` to avoid overflow, where z is a numpy matrix. This can be similarly implemented for other packages such as the `math` package at your own discretion. The argument `axis` in `np.max` may also be adjusted depending on your method of calculation. If your implementation of softmax_regression_hypothesis does not take care of numerical overflow correctly, you can still get full credit in this question, but it might affect your training of the Softmax Classifier for the later part of this notebook.


In [3]:
def softmax_regression_hypothesis(X, weights):
    '''
    Computes the hypothesis of the softmax regression model using a given input X, output a vector containing class possibilities 
    
    Parameters
    ----------
        - X : matrix containing data features (numpy array of shape (m,n))
        - weight : model parameters (numpy array of shape (n,K))
        
    Returns
    -------
        - hypothesis : resulting matrix containing probability value of different classes for input x (numpy array of shape (m,K))
    '''
    
    """
    *Important*
    
    In your calculation of the exponential term using numpy, please implement np.exp(z - np.max(z, axis, keepdims=True)) in order
    to prevent numerical overflow during training"
    """
    
    e_x = np.exp(X@weights - np.max(X@weights, axis=1, keepdims=True) ) # - np.max(X@weights, axis=1, keepdims=True) is added for numerical stability
    return (e_x.T / np.sum(e_x,axis=1)).T

In [4]:
# example execution on sample data

test_x_1 = np.array([[1, 2, 3], [4, 5, 6]])
test_weights_1 = np.array([[.1, .2], [.3, .4], [.5, .6]])
print(softmax_regression_hypothesis(test_x_1, test_weights_1))
# np.array([[0.35434369, 0.64565631],
#           [0.18242552, 0.81757448]])

test_x_2 = np.array([[6, 5], [4, 3], [2, 1], [0, 1], [1, 2]])
test_weights_2 = np.array([[.1, .2, .1, .6, .5], [.3, .2, .2, .1, .2]])
print(softmax_regression_hypothesis(test_x_2, test_weights_2))
# np.array([[0.05957114, 0.06583629, 0.03613172, 0.44017449, 0.39828635],
#           [0.09460303, 0.10455252, 0.07008365, 0.38363421, 0.34712659],
#           [0.13794432, 0.15245205, 0.12481718, 0.30700072, 0.27778574],
#           [0.22059263, 0.19960047, 0.19960047, 0.18060597, 0.19960047],
#           [0.19801424, 0.17917069, 0.16212035, 0.21883958, 0.24185514]])

[[0.35434369 0.64565631]
 [0.18242552 0.81757448]]
[[0.05957114 0.06583629 0.03613172 0.44017449 0.39828635]
 [0.09460303 0.10455252 0.07008365 0.38363421 0.34712659]
 [0.13794432 0.15245205 0.12481718 0.30700072 0.27778574]
 [0.22059263 0.19960047 0.19960047 0.18060597 0.19960047]
 [0.19801424 0.17917069 0.16212035 0.21883958 0.24185514]]


### B. Loss function [3 marks]

In order to train your softmax regression model, we will implement a generalised form of the cost function which we used to train our logistic regression model. 

We define our new cost function as follows:

$$ J(w) = - \frac{1}{m}\left[ \sum_{i=1}^{m} \sum_{k=1}^{K} \mathbb{1}_{\left\{y^{(i)} = k\right\}}  \log \frac{\exp(w^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(w^{(j)\top} x^{(i)})}\right] $$

where $\mathbb{1}_{\left\{y^{(i)} = k\right\}}$ refers to an indicator function that returns a value of 1 if $y^{(i)} = k$ and zero if $y^{(i)} \neq k$.

To allow numerical stability during training, please implement `np.log(z + eps)` in your calculation of the log term using numpy instead of `np.log(z)`.

Define a function `loss_fn` which implements this loss function. 

In [5]:
def loss_fn(X, weights, y_true):
    '''
    Loss function to be used for training of the Softmax Regression model
    
    Parameters
    ----------
        - X : data matrix (numpy array of shape (m,n))
        - weights : model parameters (numpy array of shape (n,K))
        - y_true : matrix containing one-hot encoded K ground truth targets  (numpy array of shape (m,K))    
    Returns
    -------
        - loss_val : calculated loss value (float) 
    '''
    
    """
    *Important*
    
    When you implement the log term using numpy, please use np.log(h + eps) instead of just np.log(h) for numerical stability during training, 
    where eps = np.finfo(np.float64).eps
    """
    eps = np.finfo(np.float64).eps
    h = softmax_regression_hypothesis(X, weights)
    return -(1/X.shape[0])*np.sum(np.multiply(y_true, np.log(h + eps)))

In [6]:
# example execution on sample data

test_x = np.array([[6, 5], [4, 3], [2, 1], [0, 1], [1, 2]])
test_weights = np.array([[.1, .2, .1, .6, .5], [.3, .2, .2, .1, .2]])
test_y = np.array([
    [0., 0., 0., 1., 0.], # class 3
    [1., 0., 0., 0., 0.], # class 0
    [0., 0., 1., 0., 0.], # class 2
    [0., 1., 0., 0., 0.], # class 1
    [0., 0., 0., 0., 1.], # class 4
])
round(loss_fn(test_x, test_weights, test_y), 4) # 1.6581

1.6581

### C. Gradient loss function [2 marks]

As we will be training our model using gradient descent, we define the gradient of our cost function as follows:
$$ \nabla_{w^{(k)}} J(w) = -\frac{1}{m} \sum_{i=1}^{m}{ \left[ x^{(i)} \left(  \mathbb{1}_{\left\{y^{(i)} = k\right\}}  - P(y^{(i)} = k | x^{(i)}; w) \right) \right]  } $$

Define a function `grad_loss_fn` which implements this gradient of our loss function. 

In [8]:
def grad_loss_fn(X, weights, y):
    '''
    Gradient of the Cost function to be used for training of the Softmax Regression model
    
    Parameters
    ----------
        - X : data matrix (numpy array of shape (m,n))
        - weight : model parameters (numpy array of shape (n,K))
        - y : matrix containing one-hot encoded K ground truth targets (numpy array of shape (m,K))
    
    Returns
    -------
        - grad : matrix containing calculated partial derivatives of Cost Function J (numpy array of shape (n,K))
    '''
    
    grad_i = -(1/X.shape[0])*np.dot(X.T, y-softmax_regression_hypothesis(X,weights))
    return grad_i

In [9]:
# example execution on sample data

test_x = np.array([[6, 5], [4, 3], [2, 1], [0, 1], [1, 2]])
test_weights = np.array([[.1, .2, .1, .6, .5], [.3, .2, .2, .1, .2]])
test_y = np.array([
    [0., 0., 0., 1., 0.], # class 3
    [1., 0., 0., 0., 0.], # class 0
    [0., 0., 1., 0., 0.], # class 2
    [0., 1., 0., 0., 0.], # class 1
    [0., 0., 0., 0., 1.], # class 4
])
print(grad_loss_fn(test_x, test_weights, test_y))
# np.array([[-0.55805163  0.25946052 -0.21822407 -0.19831503  0.71513022]
#           [-0.33275396  0.07064658  0.00791358 -0.14458781  0.3987816 ]])

[[-0.55805163  0.25946052 -0.21822407 -0.19831503  0.71513022]
 [-0.33275396  0.07064658  0.00791358 -0.14458781  0.3987816 ]]


### D. Classification [3 marks]

Now that you have the hypothesis, loss function and derivative of the loss function, next step will be to train your model.

A function that implements the gradient descent algorithm is provided for you. Function ```gradient_descent```, which is provided below, takes in your loss function, derivative of the loss function, and your data to train the classifer. You are to use this generic function for this question, and <b>please do not modify</b> the ```gradient_descent``` function for the purposes of grading.

Using your hypothesis, loss and gradient of loss functions derived in Parts A, B and C above, use the following  ```gradient_descent``` function given to train a <b>multi-class classifier</b> using Softmax Regression.



In [10]:
# Function is given to students
def gradient_descent(loss_fn, grad_loss_fn, X, y, weights=None):
    '''
    This function executes training of a linear/logistic regressor model using gradient descent.
    
    Parameters
    ----------
        - loss_fn : function that calculates loss/cost value (function)
        - grad_loss_fn : function that calculates the derivative of the loss/cost value(function)
        - X : features data matrix for training (numpy array of shape (m,n))
        - y : ground truth targets matrix for training (numpy array of shape (m,K)) 
        - lr : learning rate for updating of weights (float) 
        - max_iter : maximum number of iteration before termination of gradient descent (int)
        - eps : tolerance value for early stopping (float)
    Returns
    -------
        - weights : trained weights (numpy array of shape (n,K))
    '''
    
    # Constant terms. Please do not modify ============
    lr=0.01
    max_iter=2000
    eps=0.0001
    # =================================================
    
    start_time = time.time()
    
    # Initialize variables
    if weights==None:
        # xavier initialization - students need not modify this portion
        np.random.seed(2109)
        scale = 1/max(1., (2+2)/2.)
        limit = math.sqrt(3.0 * scale)
        weights = np.random.uniform(-limit, limit, size=(X.shape[1], y.shape[1]))
    last_error = np.inf
    
    for i in range(max_iter):
        current_error = loss_fn(X,weights,y)
        grad = grad_loss_fn(X,weights,y)
        weights = weights - lr * grad
        if(np.linalg.norm(current_error-last_error)<eps):
            break
        else:
            last_error = current_error

    print(f"Iterations taken: {i}, final training loss: {current_error}, Time taken: {time.time() - start_time}s")
    return weights


In particular, define a function ```softmax_regression_classification``` that implements multi-class classification using softmax regression using the weights that you have trained using gradient descent with our implementation of ```gradient_descent``` given above. **Prediction of a class of a single test data value should be chosen via max probability value**.

In [11]:
def softmax_regression_classification(X_train, y_train, X_test, loss_fn, grad_loss_fn):
    '''
    This function executes training of a softmax regressor model using gradient descent, and conducts 
    multi-class classification prediction on test data.
    
    Parameters
    ----------
        - X_train : features data matrix for training (numpy array of shape (m,n))
        - y_train : one-hot encoded ground truth targets matrix for training (numpy array of shape (m,K))
        - X_test : features data matrix for testing (numpy array of shape (z,n), where z refers to the length of test set)
        - loss_fn : function that calculates loss/cost value (function)
        - grad_loss_fn : function that calculates the derivative of the loss/cost value(function)
        
    Returns
    -------
        - y_pred : predicted classes by the model using X_test (numpy array of shape (m,))
                   Example output: array([0,0,1,2,3,4,2,3,2,...]) where classes 0 to 4 can be mapped back
                   to the dictionary `mapping` given at the start of the notebook
    '''
    trained_weights = gradient_descent(loss_fn, grad_loss_fn, X_train, y_train)
    y_pred = softmax_regression_hypothesis(X_test, trained_weights)
    return np.argmax(y_pred, axis=1)

In [12]:
# Note to students : Run this to train your model and predict on X_test.

y_pred = softmax_regression_classification(X_train, y_train, X_test, loss_fn, grad_loss_fn)
print(y_pred.shape) # (4000,)
print(y_pred) # predicted classes e.g. [2 3 2 ... 4 0 1]

# Note: The public test case on Coursemology for this part checks if your `softmax_regression_classification`
#       produces the expected predictions using sample `X_train`, `y_train`, `X_test` values, 
#       and correct implementations of `loss_fn` and `grad_loss_fn`.

Iterations taken: 492, final training loss: 1.0286823933260119, Time taken: 3.956742286682129s
(4000,)
[2 3 2 ... 0 0 2]


### E. Performance metrics [5 Marks]

Now that you have trained our model, it is important that you analyze and understand the performance of your model when tested on unseen data.

####  E.1 [2 Marks]

Define a function ```confusion_matrix``` that will calculate confusion matrix for $n$ classes. An example of a confusion matrix is as shown belows.
```
                            Predicted
     
                      class 1  class 2  class 3  class 4  class 5
          class 1   [[   x        x        x        x        x   ]
 
Actual    class 2    [   x        x        x        x        x   ]

          class 3    [   x        x        x        x        x   ]

          class 4    [   x        x        x        x        x   ]

          class 5    [   x        x        x        x        x   ]]

```

As a reminder, the confusion matrix should have True Positive values at its diagonal.

In [13]:
def confusion_matrix(y_test, y_pred):
    '''
    This function calculates the confusion matrix for any number of class n
    
    Parameters
    ----------
        - y_test : Testing ground truth targets (numpy array of shape (m,))
        - y_pred : Predicted targets by the multiclass classifier (numpy array of shape (m,))
        
    Returns
    -------
        - cm : nxn confusion matrix, which n is the number of classes (numpy array)
    '''
    num_classes = len(np.unique(y_test))
    cm = np.zeros((num_classes,num_classes))
    for i in range(len(y_test)):
        cm[y_test[i]][y_pred[i]] +=1
    return cm

In [14]:
# Sample execution

np.set_printoptions(suppress=True)

sample_y_test = np.array([0, 0, 0, 1, 2, 3, 4])
sample_y_pred = np.array([4, 0, 4, 1, 2, 3, 4]) # predicts position 0 and 2 wrongly
sample_confusion_matrix = confusion_matrix(sample_y_test, sample_y_pred)
print(sample_confusion_matrix)
# [[1. 0. 0. 0. 2.]
#  [0. 1. 0. 0. 0.]
#  [0. 0. 1. 0. 0.]
#  [0. 0. 0. 1. 0.]
#  [0. 0. 0. 0. 1.]]

[[1. 0. 0. 0. 2.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


####  E.2. [3 Marks]

We will be using a few key metrics to determine the performance of our model, which are the following:
* Precision
* Recall
* F-beta

F-beta score is an extension to the F1 Score, which is a **weighted** harmonic mean of precision and recall - meaning that now you are able to assign weights depending on the importance of precision vs recall. The general formula for non-negative real $\beta$ is given as follows:
$$ F_{\beta} = \frac{(1+\beta^{2}) \cdot (precision \cdot recall)}{(\beta^{2} \cdot precision) + recall}$$ 

A smaller beta value, such as 0.5, gives more weight to precision and less to recall, whereas a larger beta value, such as 2.0, gives less weight to precision and more weight to recall in the calculation of the score. F1 score is a case of F-beta where beta equals to 1.

Define a function `calculate_metrics` which calculates the values of precision, recall, and F-beta scores for each class by using the y_pred returned from the function that you have derived from part **D**.

In [15]:
def calculate_metrics(cm, beta=1):
    '''
    Calculates metrics using the confusion matrix
    
    Parameters
    ----------
        -  cm : nxn confusion matrix, which n is the number of classes (numpy array)
        
    Returns
    -------
        - precision : Array containing precision scores for class 0, ... , class n (Array)
            - Eg. [precision_class1, ... , precision_class5]
        - recall : Array containing recall scores for class 0, ... , class n (Array)
            - Eg. [recall_class1, ... , recall_class5]
        - f-beta : Array containing F-beta scores for class 0, ... , class n (Array)
            - Eg. [F_beta_class1, ..., F_beta_class5]
    '''
    eps = np.finfo(np.float64).eps
    precision = np.diag(cm) / np.sum(cm, axis = 0)
    recall = np.diag(cm) / np.sum(cm, axis = 1)
    fbeta = (1+(beta**2))*(np.multiply(precision, recall))/(np.add((beta**2)*precision, recall)+eps)
    return precision, recall, fbeta

In [16]:
# Sample execution

sample_confusion_matrix = np.array([
    [1., 0., 0., 0., 2.],
    [0., 1., 0., 0., 0.],
    [0., 0., 1., 0., 0.],
    [0., 0., 0., 1., 0.],
    [0., 0., 0., 0., 1.],
])
sample_precision, sample_recall, sample_fbeta = calculate_metrics(sample_confusion_matrix, beta=1)
print(sample_precision)
print(sample_recall)
print(sample_fbeta)
# dummy metrics for test case:
# class:      0          1          2          3          4
# precision: [1.         1.         1.         1.         0.33333333]
# recall:    [0.33333333 1.         1.         1.         1.        ]
# fbeta:     [0.5        1.         1.         1.         0.5       ]

[1.         1.         1.         1.         0.33333333]
[0.33333333 1.         1.         1.         1.        ]
[0.5 1.  1.  1.  0.5]


In [17]:
# actual results
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
precision, recall, fbeta = calculate_metrics(cm)
print(precision)
print(recall)
print(fbeta)

[0.82063264 0.         0.86704653 0.03184713 0.        ]
[0.90023659 0.         0.68646617 0.14705882 0.        ]
[0.85859346 0.         0.76626102 0.05235602 0.        ]


### F. Evaluate and explain [4 marks]

Based on the metrics that you have calculated, evaluate your multi-class classifier's performance by answering the following questions in this section.

You should provide justifications in relation to the context / data / method wherever necessary, and answer the questions in a **short and concise manner**. 

We will be awarding marks based on your understanding of metrics, and logical application of those metrics to interpret your results. 

[4 Marks]

#### F.1

Is precision *and/or* recall a good measure of performance for an IDS system? 
Please explain your answer in less than **3 concise sentences**..
[2 Marks]

#### F.2

Explain your model performance scores for the different types of attacks with respect to the precision/recall/F-beta scores which you have calculated. Explain your choice of the beta value as well. Please keep your answer to less than **3 concise sentences**.

[2 Marks]